In [13]:
#for loading files
import os
import glob
import pandas as pd
import numpy as np

Assuptions:

Modifications:
 - df_TOC
     - consolidated actual and budgeted operation costs
     - set all dates to first in month
     - filled values for missing dates mean of three previous months)
 - df_employees

 - df_accounts

 - df_org


In [93]:
# ----------- Loading data ----------
# Operational cost data
path_TOC = os.getcwd() + "/DTO/Total_Operating_Costs/Actual_Costs" #Find the csv files in the working directory, 
files_TOC = glob.glob(path_TOC + "/*.csv") #loop through all csv files in folder
df_TOC = pd.concat(map(lambda file: pd.read_csv(file, sep=";"), files_TOC)) #read as panda dataframes and concatenate.

# Future data
path_future_TOC = os.getcwd() + "/DTO/Total_Operating_Costs/Future_Budget/DTO_TOC_FUT_2020_2H.csv"
df_future_TOC = pd.read_csv(path_future_TOC, sep=";")

# Employees data
path_employees = os.getcwd() + "/DTO/Employee_data/DTO_EMPLOYEES_REA_2018_C.csv"
df_employees = pd.read_csv(path_employees, sep=",")

# Account data
path_accounts = os.getcwd() + "/Master_Data/DIM_MASTER_ACCOUNTGROUPS.csv"
df_accounts = pd.read_csv(path_accounts, sep=";")

# Organisation data
path_organisation = os.getcwd() + "/Master_Data/DIM_MASTER_ORGANISATION.csv"
df_org = pd.read_csv(path_organisation, sep=";")

# ----------- Preparing data ----------
#joining actual and future data, add column to indicate source
df_TOC['Source'] = "actual"
df_future_TOC['Source'] = "future"
df_TOC = pd.concat([df_TOC, df_future_TOC])

#renaming
df_TOC = df_TOC.rename(index=str, columns={"Account number": "AccountNumber", "Value DKK": "ValueDKK", "Cost Center":"CostCenter"})
df_accounts = df_accounts.rename(index=str, columns={"Account 2-digits": "AccountCode", "Account Type":"AccountType", "Account Group":"AccountGroup"})
df_org = df_org.rename(index=str, columns={"Location Code": "LocationCode", "Department Number": "DepartmentNumber", "Cost Center":"CostCenter"})

#strip strings for meaningful data (Account Number, ), remove columns, add dates column
df_TOC['AccountCode'] = np.int64(df_TOC.AccountNumber.str[5:7])
df_TOC['Year'] = np.int64(df_TOC.Year.str[3::])
df_TOC['Date'] = "1-" + df_TOC.Month.astype(str) + "-" + df_TOC.Year.astype(str)
df_TOC['Date'] = pd.to_datetime(df_TOC['Date'], format='%d-%m-%Y')
df_TOC = df_TOC.drop(['AccountNumber', 'Year', 'Month'], axis=1)


#duplicates in master data (organisation)
df_org = df_org.drop(df_org[df_org["CostCenter"].duplicated()].index)

#check if we have all cost centeres
a = df_TOC.CostCenter.unique().tolist()
b = df_org.CostCenter.unique().tolist()
all(item in a for item in b)

#changing type to allow mathematical operations
df_TOC["ValueDKK"]=df_TOC["ValueDKK"].str.replace(',','.')
df_TOC["ValueDKK"]= df_TOC["ValueDKK"].astype(float).astype(int) #loses some precision but allows easier operations

#find and fill avg of three months before missing date, merged with TOC data.
missing_dates = pd.date_range(start = '2018-01-02', end = '2020-12-01', freq='MS').difference(df_TOC["Date"]) #find missing dates
for d in missing_dates:
    df = df_TOC.loc[(df_TOC['Date'] >= (d - pd.DateOffset(months=3))) & (df_TOC['Date'] < d)] #gets all rows of three months previous of selected date
    df = df.groupby(['Date', 'CostCenter', "AccountCode"]).sum().ValueDKK.astype(int).reset_index() #gets average over cost centeres and account codes
    df = df.groupby(['CostCenter', 'AccountCode']).mean().ValueDKK.astype(int).reset_index()
    df['Source'], df['Date'] = 'imputed', d
    df_TOC = pd.concat([df_TOC, df]).sort_values('Date')

display(df_TOC.groupby(['Date']).sum().ValueDKK.astype(int).reset_index())
#test consistency of TOC data after fill

# ----------- Displaying data ----------
display(df_TOC)
#display(df_employees)
#display(df_accounts)
#display(df_org)

#------------ Export data -----------
df_TOC.to_csv('TOC_transformed.csv')
#df_employees.to_csv('employees_transformed.csv')
#df_accounts.to_csv('accounts_transformed.csv')
#df_org.to_csv('org_transformed.csv')

,Date,ValueDKK
0,2018-01-01,76357865
1,2018-02-01,37963830
2,2018-03-01,941036496
3,2018-04-01,762384992
4,2018-05-01,793532633
5,2018-06-01,793667664
6,2018-07-01,837860466
7,2018-08-01,859453847
8,2018-09-01,890629535
9,2018-10-01,859356106


,ValueDKK,CostCenter,Source,AccountCode,Date
32768,6070,20,actual,24,2018-01-01
10439,33353,394,actual,22,2018-01-01
58313,4895,663,actual,38,2018-01-01
58307,38014,663,actual,38,2018-01-01
10434,45464,394,actual,21,2018-01-01
...,...,...,...,...,...
4304,2356,358,future,22,2020-12-01
26774,92,224,future,24,2020-12-01
62871,21584,527,future,23,2020-12-01
26222,3237,2,future,24,2020-12-01


In [15]:
### Task 1:


In [4]:
### Task 2:

# ----------- Preparing data ----------
#renaming
df_employees = df_employees.rename(index=str, columns={"Cost Center":"CostCenter"})

# change the format of the table
employees_unp = df_employees.melt(id_vars=['CostCenter'], var_name='Date', value_name='noEmployees')

# format the data
employees_unp['Date'] = pd.to_datetime(employees_unp['Date'], format='%m/%d/%Y') #change date format
employees_unp['noEmployees'] = employees_unp['noEmployees'].fillna(0) # fill with 0

#check whether there are negative values and change to 0
employees_unp['noEmployees'] = employees_unp['noEmployees'].astype(int)
employees_unp.at[employees_unp[employees_unp['noEmployees']<0].index.values, 'noEmployees'] = 0

#How many  employees work for the Organisation
print(employees_unp["noEmployees"].sum()) #247997 people

n_employees = employees_unp["noEmployees"][employees_unp["Date"] == "2020-06-01"].sum() #8991 employees
print("Number of employees in 2020-06-01: {0}".format(n_employees)) 

# add missing dates and values
missing_dates3 = pd.date_range(start = '2018-01-02', end = '2020-06-01', freq='MS').difference(employees_unp["Date"]) #find missing dates
un_cc = pd.unique(employees_unp["CostCenter"]) #take unique values of costcenters
for i in missing_dates3:
    for j in un_cc:
        employees_unp = employees_unp.append({"CostCenter": j, "Date": i, "noEmployees": np.nan}, ignore_index=True) #add new raws with new dates for all cc
empl_cc = employees_unp.sort_values(by = ["CostCenter", "Date"]) #sort by cc and date
empl_cc.interpolate(method='linear', inplace=True) #interpolated linearly
empl_cc.to_csv("full_emp_data.csv")

# ----------- Displaying data ----------
display(empl_cc)

247997
Number of employees in 2020-06-01: 8991


,CostCenter,Date,noEmployees
191,1,2018-01-01,2.0
694,1,2018-02-01,2.0
1197,1,2018-03-01,2.0
1700,1,2018-04-01,2.0
2203,1,2018-05-01,2.0
...,...,...,...
10977,673,2020-02-01,33.0
11480,673,2020-03-01,33.0
11983,673,2020-04-01,33.0
12486,673,2020-05-01,36.0


In [5]:
### Task 3:

# ----------- Preparing data ----------
# remove spaces from each column if it is a string type
for col in df_accounts.columns:
    if df_accounts.dtypes[col] == object:
        for index in df_accounts.index:
            df_accounts.at[index, col] = df_accounts[col][index].strip()
            
# merging tables with employess and organisations
emp_org = pd.merge(employees_unp, df_org, on='CostCenter', how='inner') #tables with employess and organisations
print(emp_org.shape)
print(employees_unp.shape) #check that number of records are the same

# merging account data and costs data
TOC_acc = pd.merge(df_TOC, df_accounts, on = "AccountCode", how = "left") 

print(TOC_acc.shape)
print(df_TOC.shape) #check that number of records are the same. 
# Appeared that there are not because we have accCode 33 that is not in Master data. Assume that it is "other costs"
TOC_acc[TOC_acc.isnull().any(axis=1)]

TOC_acc_p = TOC_acc[TOC_acc["AccountGroup"] == "People"] #take only costs related to people

# creating new table that is group people costs data by CostCenter and Date to merge it with people data
grouped_TOC = TOC_acc_p.groupby(["CostCenter", "Date"]).sum() 

# merging all data together 
all_employee = pd.merge(grouped_TOC, emp_org, on = ["Date", "CostCenter"], how = "inner")

#create a column cost per employee for year data and fpr year + cost center data
# all_employee_date = all_employee.groupby("Date").sum()
# all_employee_date["CostEmployee"] = all_employee_date["ValueDKK"]/ all_employee_date["noEmployees"]

# all_employee_cc = all_employee.groupby(["Date", "CostCenter"]).sum()
# all_employee_cc["CostEmployee"] = all_employee_cc["ValueDKK"]/ all_employee_cc["noEmployees"]

# ----------- Displaying data ----------
display(all_employee)

(15090, 7)
(15090, 3)
(478830, 10)
(478830, 8)


,Date,CostCenter,Year,Month,ValueDKK,AccountCode,noEmployees,LocationCode,Location,Company,DepartmentNumber
0,2018-01-01,1,26234,13,1918890,282,2.0,2000,Denmark,Company B,79_9
1,2018-02-01,1,20180,20,1523598,217,2.0,2000,Denmark,Company B,79_9
2,2018-03-01,1,18162,27,1518590,196,2.0,2000,Denmark,Company B,79_9
3,2018-04-01,1,14126,28,1612954,152,2.0,2000,Denmark,Company B,79_9
4,2018-05-01,1,14126,35,1576990,152,2.0,2000,Denmark,Company B,79_9
...,...,...,...,...,...,...,...,...,...,...,...
11233,2020-02-01,673,38380,38,1793460,406,33.0,7000,Denmark,Company A,98_4
11234,2020-03-01,673,38380,57,1793460,406,33.0,7000,Denmark,Company A,98_4
11235,2020-04-01,673,38380,76,1808078,406,33.0,7000,Denmark,Company A,98_4
11236,2020-05-01,673,38380,95,2007106,406,36.0,7000,Denmark,Company A,98_4
